# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [47]:
# !pip install xlsxwriter

import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
IEX_CLOUD_API_TOKEN =  'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'avg10Volume': 121304243,
 'avg30Volume': 118948506,
 'beta': 1.1944181177554232,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 117.9,
 'day30ChangePercent': 0.02317133884363551,
 'day50MovingAvg': 131.35,
 'day5ChangePercent': -0.055405796422063516,
 'dividendYield': 0.00636305131047076,
 'employees': 0,
 'exDividendDate': '2020-10-25',
 'float': 0,
 'marketcap': 2195724905131,
 'maxChangePercent': 49.24906902263856,
 'month1ChangePercent': 0.03719680629091061,
 'month3ChangePercent': 0.12166761451656469,
 'month6ChangePercent': 0.375376570666674,
 'nextDividendDate': '0',
 'nextEarningsDate': '0',
 'peRatio': 38.75057295664879,
 'sharesOutstanding': 17210180019,
 'ttmDividendRate': 0.8328195666263073,
 'ttmEPS': 3.32,
 'week52change': 0.704123067095002,
 'week52high': 142.01,
 'week52low': 56.39,
 'year1ChangePercent': 0.7040298139054849,
 'year2ChangePercent': 2.5074022457897476,
 'year5ChangePercent': 4.464717264523358,
 'ytdChangePercent': -0.04711592325405652}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
# for symbol in stocks:
#   api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
#   data = requests.get(api_url).json()
  

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

# /stock/{symbol}/batch
for symbol_string in symbol_strings :
  batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price,quote&token={IEX_CLOUD_API_TOKEN}"
  data  = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(","):
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            "N/A"],
            index = my_columns),
      ignore_index = True
    )

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,128.01,0.478394,N/A
1,AAL,15.61,-0.442005,N/A
2,AAP,167.03,0.0686068,N/A
3,AAPL,132.20,0.72577,N/A
4,ABBV,108.21,0.256662,N/A
...,...,...,...,...
500,YUM,106.94,0.0657693,N/A
501,ZBH,168.20,0.0926691,N/A
502,ZBRA,406.94,0.541096,N/A
503,ZION,48.60,-0.0147146,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [ ]:
final_dataframe.sort_values('One-Year Price Return',ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
print(final_dataframe)

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [18]:
def portfolio_input():
  global portfolio_size
  try:
    portfolio_size = float(input("Enter the amount to Trade: "))
  except ValueError:
    print("Invalid Input")
    portfolio_size = float(input("Enter the amount to Trade: "))

  return portfolio_size
portfolio_input()

Enter the amount to Trade: 1000000


1000000.0

In [ ]:
position_size = float(portfolio_size/len(final_dataframe.index))
for i in range(len(final_dataframe)):
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, "Price"])

final_dataframe

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [43]:
hqm_column = [
      "Ticker",
      "Price",
      "Number of Shares To Buy",
      "One-Year Price Return",
      "One-Year Return Percentile",
      "Six-Month Price Return",
      "Six-Month Return Percentile",
      "Three-Month Price Return",
      "Three-Month Return Percentile",
      "One-Month Price Return",
      "One-Month Return Percentile",
      "HQM-Score"]

hqm_dataframe = pd.DataFrame(columns= hqm_column)

for symbol_string in symbol_strings:
  batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price,quote&token={IEX_CLOUD_API_TOKEN}"
  data  = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(","):
    hqm_dataframe = hqm_dataframe.append(
        pd.Series(
            [
            symbol,
            data[symbol]['price'],
            "N/A",
            data[symbol]['stats']['year1ChangePercent'],
            "N/A",
            data[symbol]['stats']['month6ChangePercent'],
            "N/A",
            data[symbol]['stats']['month3ChangePercent'],
            "N/A",
            data[symbol]['stats']['month1ChangePercent'],
            "N/A",
            "N/A"
            ],
            index = hqm_column),
      ignore_index = True
    )

hqm_dataframe


,Ticker,Price,Number of Shares To Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM-Score
0,A,130.120,N/A,0.472694,N/A,0.389709,N/A,0.216787,N/A,0.0705587,N/A,N/A
1,AAL,16.324,N/A,-0.431336,N/A,0.222496,N/A,0.244887,N/A,-0.0540253,N/A,N/A
2,AAP,167.641,N/A,0.0709316,N/A,0.201073,N/A,0.0766572,N/A,0.0923178,N/A,N/A
3,AAPL,134.700,N/A,0.721735,N/A,0.375848,N/A,0.126376,N/A,0.0368598,N/A,N/A
4,ABBV,108.205,N/A,0.252099,N/A,0.0984202,N/A,0.251457,N/A,-0.0158535,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.900,N/A,0.067399,N/A,0.217497,N/A,0.163853,N/A,0.00520867,N/A,N/A
501,ZBH,161.220,N/A,0.094004,N/A,0.366596,N/A,0.138368,N/A,0.0790112,N/A,N/A
502,ZBRA,414.940,N/A,0.545603,N/A,0.523471,N/A,0.471477,N/A,0.0440438,N/A,N/A
503,ZION,53.080,N/A,-0.0145981,N/A,0.511579,N/A,0.599021,N/A,0.104185,N/A,N/A


In [88]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [89]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
  for time_period in time_periods:
    hqm_dataframe.loc[row, f"{time_period} Return Percentile"] = stats.percentileofscore(hqm_dataframe[f"{time_period} Price Return"],hqm_dataframe.loc[row, f"{time_period} Price Return"])/100

hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.480,N/A,0.473189,0.879208,0.382315,0.724752,0.213954,0.615842,0.0717034,0.794059,N/A
1,AAL,16.173,N/A,-0.432627,0.019802,0.221726,0.465347,0.246394,0.657426,-0.054215,0.0831683,N/A
2,AAP,171.930,N/A,0.0696208,0.459406,0.202288,0.421782,0.0749042,0.310891,0.0953753,0.881188,N/A
3,AAPL,132.690,N/A,0.707474,0.960396,0.36959,0.706931,0.126276,0.429703,0.0370947,0.590099,N/A
4,ABBV,107.720,N/A,0.247691,0.691089,0.0970866,0.249505,0.257213,0.681188,-0.0160937,0.259406,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.310,N/A,0.0657491,0.449505,0.21569,0.453465,0.169083,0.518812,0.00522931,0.39604,N/A
501,ZBH,163.010,N/A,0.0944864,0.49505,0.368411,0.70099,0.139861,0.445545,0.0798533,0.817822,N/A
502,ZBRA,428.820,N/A,0.538579,0.914851,0.51744,0.861386,0.475202,0.90297,0.0440893,0.635644,N/A
503,ZION,52.700,N/A,-0.0147598,0.306931,0.523827,0.865347,0.605468,0.958416,0.102798,0.90099,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [90]:
from statistics import mean

for row in hqm_dataframe.index:
  momentum_percentile = []
  for time_period in time_periods:
    momentum_percentile.append(hqm_dataframe.loc[row, f"{time_period} Return Percentile"])
  hqm_dataframe.loc[row, "HQM Score"] = mean(momentum_percentile)

hqm_dataframe 

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.480,N/A,0.473189,0.879208,0.382315,0.724752,0.213954,0.615842,0.0717034,0.794059,0.753465
1,AAL,16.173,N/A,-0.432627,0.019802,0.221726,0.465347,0.246394,0.657426,-0.054215,0.0831683,0.306436
2,AAP,171.930,N/A,0.0696208,0.459406,0.202288,0.421782,0.0749042,0.310891,0.0953753,0.881188,0.518317
3,AAPL,132.690,N/A,0.707474,0.960396,0.36959,0.706931,0.126276,0.429703,0.0370947,0.590099,0.671782
4,ABBV,107.720,N/A,0.247691,0.691089,0.0970866,0.249505,0.257213,0.681188,-0.0160937,0.259406,0.470297
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.310,N/A,0.0657491,0.449505,0.21569,0.453465,0.169083,0.518812,0.00522931,0.39604,0.454455
501,ZBH,163.010,N/A,0.0944864,0.49505,0.368411,0.70099,0.139861,0.445545,0.0798533,0.817822,0.614851
502,ZBRA,428.820,N/A,0.538579,0.914851,0.51744,0.861386,0.475202,0.90297,0.0440893,0.635644,0.828713
503,ZION,52.700,N/A,-0.0147598,0.306931,0.523827,0.865347,0.605468,0.958416,0.102798,0.90099,0.757921


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [99]:
hqm_dataframe.sort_values("HQM Score", ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50].reset_index()

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
portfolio_input()
position_size = float(portfolio_size/len(hqm_dataframe))

for row in hqm_dataframe.index:
  hqm_dataframe.loc[row, "Number of Shares to Buy"] = math.floor(position_size/hqm_dataframe.loc[row, "Price"])

hqm_dataframe

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [103]:
writer = pd.ExcelWriter("momentum_strategy.xlsx", engine = "xlsxwriter")

hqm_dataframe.to_excel(writer, sheet_name="Momentum Startegy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [104]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [106]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
  writer.sheets["Momentum Startegy"].set_column(f"{column}:{column}", 22, column_formats[column][1])

writer.save()

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [ ]:
writer.save()